# **CNN LeNet-5**

#### Import libraries

In [1]:
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from statistics import mean

#### General configurations

In [2]:
np.random.seed(1)
tf.random.set_seed(1)

In [3]:
def to_binary_categorical(array):
    new_array = []

    for i in range(len(array)):
        content = [0] * 25
        index = np.argmax(array[i])
        content[index] = 1
        new_array.append(content)

    return new_array
    

#### Initialise k-fold

In [4]:
kf = StratifiedKFold(n_splits=5)

#### Load and prepare data

In [5]:
train_data = np.genfromtxt('../dataset/sign_mnist_train.csv', delimiter=',')
test_data = np.genfromtxt('../dataset/sign_mnist_test.csv', delimiter=',')

In [6]:
X_train = train_data[1:, 1:]
y_train = train_data[1:, 0]

X_test = test_data[1:, 1:]
y_test = test_data[1:, 0]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(27455, 784) (27455,)
(7172, 784) (7172,)


#### Preprocess data

In [7]:
# normalise pixel values from 0-255 to 0-1
X_train = X_train / 255.0
X_test = X_test / 255.0

# reshape 1d array of 784 to 2d array of size 28x28
X_train = X_train.reshape(len(X_train), 28, 28)
X_test = X_test.reshape((len(X_test)), 28, 28)

print(X_train.shape)
print(X_test.shape)

(27455, 28, 28)
(7172, 28, 28)


#### LeNet-5 Model

In [8]:
model = Sequential()

model.add(Conv2D(input_shape=(28, 28, 1), kernel_size=(5, 5), filters=6, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(kernel_size=(5, 5), filters=16, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Flatten())

model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(units=25, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_crossentropy'])

2022-04-21 18:45:34.005492: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Validation using k-fold

In [9]:
f1_scores = []

for train_index, test_index in kf.split(X_train, y_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # one-hot
    y_train_fold = to_categorical(y_train_fold)
    y_test_fold = to_categorical(y_test_fold)

    # fit and predict
    model.fit(x=X_train_fold, y=y_train_fold)
    y_predicted = model.predict(x=X_test_fold)

    # convert predicted float values to binary values
    y_predicted = to_binary_categorical(y_predicted)
    
    # calculate score
    score = f1_score(y_test_fold, y_predicted, average='micro')
    f1_scores.append(score)

print(f1_scores)
print('Average F1 Score:', mean(f1_scores))

687/687 [==============================] - 8s 12ms/step - loss: 0.0333 - categorical_crossentropy: 0.0333
[0.7907484975414314, 0.9635767619741397, 0.9344381715534511, 0.9998178838098707, 0.9901657257330176]
Average F1 Score: 0.935749408122382


#### Final Result 

In [10]:
y_test = to_categorical(y_test)

y_predicted = model.predict(x=X_test)
y_predicted = to_binary_categorical(y_predicted)

score = f1_score(y_test, y_predicted, average='micro')
print('F1 Score: ', score)

F1 Score:  0.8350529838259899
